In [18]:
import pandas as pd
import itertools 
import string
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
#import matplotlib.pyplot as plt
#%matplotlib inline
#pd.set_option('display.max_rows', None)

In [ ]:
import pandas as pd
import string
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [4]:
X = pd.read_csv('filtered_df.csv')
X.head()

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
0,75612,L-methylfolate,Depression,"""I have taken anti-depressants for years, with...",10,"March 9, 2017",54
1,96233,Sertraline,Depression,"""1 week on Zoloft for anxiety and mood swings....",8,"May 7, 2011",3
2,121333,Venlafaxine,Depression,"""my gp started me on Venlafaxine yesterday to ...",4,"April 27, 2016",3
3,156544,Dulaglutide,"Diabetes, Type 2","""Hey Guys, It&#039;s been 4 months since my l...",10,"October 24, 2017",24
4,131909,Effexor XR,Depression,"""This medicine saved my life. I was at my wits...",10,"June 20, 2013",166


In [5]:
X1 = X.drop(['Unnamed: 0','drugName','rating','date','usefulCount'],axis=1)
X1.head()

,condition,review
0,Depression,"""I have taken anti-depressants for years, with..."
1,Depression,"""1 week on Zoloft for anxiety and mood swings...."
2,Depression,"""my gp started me on Venlafaxine yesterday to ..."
3,"Diabetes, Type 2","""Hey Guys, It&#039;s been 4 months since my l..."
4,Depression,"""This medicine saved my life. I was at my wits..."


In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
import nltk
from nltk.corpus import stopwords

stop = stopwords.words('english')
stop

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [8]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [9]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [10]:
from bs4 import BeautifulSoup
import re

In [11]:
def review_to_words(raw_review):
    # 1. Delete HTML 
    review_text = BeautifulSoup(raw_review, 'html.parser').get_text()
    # 2. Make a space
    letters_only = re.sub('[^a-zA-Z]', ' ', review_text)
    # 3. lower letters   
    words = letters_only.lower().split()
    # 5. Stopwords 
    meaningful_words = [w for w in words if not w in stopwords]
    # 6. lemmitization
    lemmitize_words = [lemmatizer.lemmatize(w) for w in meaningful_words]
    # 7. space join words
    return( ' '.join(lemmitize_words))

In [12]:
X1['review_clean'] = X1['review'].apply(review_to_words)
X1.head()

<ipython-input-11-96d5791fc591>:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(raw_review, 'html.parser').get_text()


,condition,review,review_clean
0,Depression,"""I have taken anti-depressants for years, with...",taken anti depressant year improvement mostly ...
1,Depression,"""1 week on Zoloft for anxiety and mood swings....",week zoloft anxiety mood swing take mg morning...
2,Depression,"""my gp started me on Venlafaxine yesterday to ...",gp started venlafaxine yesterday help depressi...
3,"Diabetes, Type 2","""Hey Guys, It&#039;s been 4 months since my l...",hey guy month since last post wanted give mont...
4,Depression,"""This medicine saved my life. I was at my wits...",medicine saved life wit end anti depressant re...


In [13]:
X_feat=X1['review_clean']
y=X1['condition']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_feat, y,stratify=y,test_size=0.2, random_state=0)

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.8)
tfidf_train = tfidf_vectorizer.fit_transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)                         #tfidf_vectorizer    rf_tf = RandomForestClassifier()

In [16]:
rf_tf = RandomForestClassifier() 
rf_tf.fit(tfidf_train, y_train)

RandomForestClassifier()

In [19]:
pred1 = rf_tf.predict(tfidf_test)
score1 = metrics.accuracy_score(y_test, pred1)
print("accuracy:   %0.5f" % score1)

accuracy:   0.96701


In [20]:
from sklearn.metrics import classification_report
print(classification_report(y_test,pred1))

                     precision    recall  f1-score   support

         Depression       0.96      1.00      0.98      1814
   Diabetes, Type 2       0.99      0.94      0.96       511
High Blood Pressure       0.99      0.88      0.93       464

           accuracy                           0.97      2789
          macro avg       0.98      0.94      0.96      2789
       weighted avg       0.97      0.97      0.97      2789



In [22]:
test = tfidf_vectorizer.transform(["Irregular heartbeat"]).toarray()  
pred=rf_tf.predict(test)[0] 
pred

'High Blood Pressure'

In [23]:
import pickle
pickle.dump(tfidf_vectorizer,open('TfidfVectorizer.pkl','wb'))
pickle.dump(rf_tf,open('model.pkl','wb'))